## Used care listings
We are working with dataset fof used cars from eBay 'Kleinanzeigen', a classfieds section of the German eBay website. The dataset was originally scraped and uploaded to Kaggle, now with a few modifications from the original dataset:

+ now sampled 50,000 data points from the full dataset to ensure the code runs quickly in hosted enviroment
+ Dirtied the dataset a bit to more closely resemable what would have been expected from a scraped dataset

The aim of this proejct is to clearn the data and analyze the included used car listings.


In [1]:
import pandas as pd

dateparser = lambda x:pd.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
autos = pd.read_csv('autos.csv',encoding='Latin-1',parse_dates=['dateCrawled','dateCreated','lastSeen'],date_parser=dateparser)
autos.head()



,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


In [2]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null datetime64[ns]
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null datetime64[ns]
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50

## Observations
+ data volume of 50000
+ memory usage of 7.6+ MB
+ 20 columns of mostly strings
+ some columns have null values, not more than 20% of the data
+ The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [4]:
autos = autos.rename(columns={'dateCrawled':'date_crawled', 
                              'yearOfRegistration':'registration_year',
                      'monthOfRegistration':'registration_month',
                      'notRepairedDamage':'unrepaired_damage',
                      'dateCreated':'ad_created',
                      'lastSeen':'last_seen',
                      'postalCode':'postal_code',
                     'nrOfPictures':'number_pictures',
                     'fuelType':'fuel_type',
                     'powerPS':'power_ps',
                     'vehicleType':'vehicle_type',
                     'offerType':'offer_type'})

In [5]:
autos.head()


,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to e more descriptive

In [6]:
autos.describe()

,registration_year,power_ps,registration_month,number_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


## Observations
+ number of pictures seem to mostly have one value that may not be relevant to our analysis
+ let's investigate more into power_ps
+ seems registration year and month might be stored as strings and may need some cleaning
+ We may also have to clean up the trailing longs into floats in each column depending on our usage

In [7]:
import re

autos.loc[:,['price','odometer']].replace(to_replace='\$([0-9,\.]+).*', value=r'\1',regex=True,inplace=True)


In [8]:
autos['price'] = [i.replace('$','').replace(',','') if '$' in i else i for i in autos['price']]
autos['price'] = autos['price'].astype('int')

In [9]:
autos['odometer'] = [i.replace('km','').replace(',','') if 'km' in i else i for i in autos['odometer']]
autos['odometer'] = autos['price'].astype('int')

In [10]:
autos[['price','odometer']].dtypes

price       int64
odometer    int64
dtype: object

In [11]:
autos = autos.rename(columns={'odometer':'odometer_km'})

In [12]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,5000,3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,8500,6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,8990,7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,4350,6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,1350,7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


In [13]:
autos[['price','odometer_km']].sort_values(by=['price','odometer_km'],ascending=False)

,price,odometer_km
39705,99999999,99999999
42221,27322222,27322222
27371,12345678,12345678
39377,12345678,12345678
47598,12345678,12345678
2897,11111111,11111111
24384,11111111,11111111
11137,10000000,10000000
47634,3890000,3890000
7814,1300000,1300000


In [14]:
autos = autos[(autos[['price','odometer_km']] > 0).all(axis=1) & (autos[['price','odometer_km']] < 169000).all(axis=1)]

# cleaning summary
+ did a sort values of both columns
+ found repeating dubious integer values, went ahead to index in to smallest range out of those values 

In [19]:
autos.sort_values(by=['date_crawled','ad_created','last_seen'],ascending=True)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_pictures,postal_code,last_seen
29899,2016-03-05 14:06:30,VW_Multivan_T4_2.5_Automatik_/_auch_Behinderte...,privat,Angebot,3950,test,bus,1995,automatik,110,transporter,3950,6,benzin,volkswagen,nein,2016-03-05,0,65719,2016-03-13 10:11:17
26440,2016-03-05 14:06:40,Seat_Alhambra_1.9_TDI,privat,Angebot,3300,test,bus,2001,manuell,116,alhambra,3300,10,diesel,seat,nein,2016-03-05,0,57577,2016-03-12 14:16:09
20681,2016-03-05 14:07:04,Schanier._Tuerschanier___hecktuerschanier_merc...,privat,Angebot,100,test,NaN,2005,NaN,0,g_klasse,100,0,NaN,mercedes_benz,NaN,2016-03-05,0,57610,2016-04-06 20:45:36
12426,2016-03-05 14:07:08,Opel_Corsa_B_1_2_L_TÜV_01.2018,privat,Angebot,750,test,NaN,2018,manuell,0,corsa,750,4,benzin,opel,nein,2016-03-05,0,53547,2016-03-06 12:33:37
9524,2016-03-05 14:07:21,Skoda_Fabia_1.2_HTP_Combi_Exact,privat,Angebot,2600,test,kombi,2003,manuell,64,fabia,2600,2,benzin,skoda,nein,2016-03-05,0,65197,2016-03-06 15:31:29
25599,2016-03-05 14:07:26,BMW_320_Cd,privat,Angebot,7249,test,coupe,2004,manuell,150,3er,7249,4,diesel,bmw,nein,2016-03-04,0,61479,2016-03-09 06:16:18
47236,2016-03-05 14:07:40,BMW_Z4_2.2i_schw._met._+_Hardtop_8_Fach_bereift,privat,Angebot,9990,test,cabrio,2005,manuell,170,z_reihe,9990,5,benzin,bmw,nein,2016-03-04,0,35519,2016-03-09 23:16:02
2469,2016-03-05 14:07:45,Volkswagen_Golf_3_mit_TÜV_bis_10/17,privat,Angebot,1450,test,kleinwagen,1996,automatik,75,golf,1450,9,benzin,volkswagen,nein,2016-03-04,0,61381,2016-03-05 15:16:47
12739,2016-03-05 14:08:00,"Chrysler_Stratus_2.5_LX_""Gepflegt""_AHK",privat,Angebot,2700,test,cabrio,1998,automatik,163,andere,2700,2,benzin,chrysler,nein,2016-03-04,0,35510,2016-03-07 17:19:55
21321,2016-03-05 14:08:00,Honda_HR_V_2wd_TÜV_NEU,privat,Angebot,2499,test,suv,2001,manuell,105,andere,2499,3,benzin,honda,nein,2016-03-04,0,57610,2016-04-06 05:16:44


In [21]:
import numpy as np
autos[autos[['date_crawled','ad_created','last_seen']].all(axis=1) == np.nan ]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_pictures,postal_code,last_seen


In [23]:
autos.loc[:,'date_crawled'].value_counts(normalize=True,dropna=False)

2016-03-08 10:40:35    0.000062
2016-03-12 16:06:22    0.000062
2016-03-09 11:54:38    0.000062
2016-04-04 16:40:33    0.000062
2016-03-23 19:38:20    0.000062
2016-03-11 22:38:16    0.000062
2016-03-30 19:48:02    0.000062
2016-03-21 20:37:19    0.000062
2016-04-02 15:49:30    0.000062
2016-03-05 16:57:05    0.000062
2016-03-25 19:57:10    0.000062
2016-03-19 17:36:18    0.000062
2016-04-02 11:37:04    0.000062
2016-03-22 09:51:06    0.000062
2016-03-29 23:42:13    0.000062
2016-03-16 21:50:53    0.000062
2016-03-14 20:50:02    0.000062
2016-03-23 18:39:34    0.000062
2016-03-31 11:42:09    0.000041
2016-03-20 15:49:05    0.000041
2016-03-20 20:42:08    0.000041
2016-03-07 17:38:17    0.000041
2016-03-20 13:39:29    0.000041
2016-03-08 17:52:23    0.000041
2016-04-04 16:25:24    0.000041
2016-03-19 13:42:48    0.000041
2016-03-17 10:37:23    0.000041
2016-03-27 19:50:57    0.000041
2016-04-01 22:49:21    0.000041
2016-03-17 21:37:43    0.000041
                         ...   
2016-03-